<hr style="border:2px solid #0176DE"> </hr>
<center><h1 style="color:#173F8A;"> Escuela de Verano en Metodos Iterativos</h1></center> 
<center><h1 style="color:#173F8A;"> EMI 2024 - CMM Chile</h1></center>
<hr style="border:2px solid #0176DE"> </hr>
<h3 style="color:#173F8A;text-align:right;"> Profesores: &nbsp;Nicolás Barnafi<br>Manuel A. Sanchez<br></h3>

<h3 style="color:#03122E;text-align:right;"> 
    Centro de Modelamiento Matematico <br> 
    Instituto de Ingenieria Matematica y Computacional - IMC UC<br>  
</h3>

<hr style="border:2px solid #03122E"> </hr>
<center><h1 style="color:#173F8A;"> Modulo 4: Precondicionadores</h1></center> 
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC:
Primaria: 
celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 
Secundaria
gris oscuro: #707070
-->

# Optimal preconditioners

$$ P^{-1}Ax = P^{-1}b $$
    
- In FEM, a preconditioner is optimal if
    $$ \rho( P_h^{-1} A_h ) < C\qquad \forall h>0 $$
- One fundamental concept to understand: *spectral equivalence*
- The best candidates for optimal preconditioners: Domain decomposition & (Algebraic) Multigrid
- For block systems, we have block factorization:
      $$ \newcommand{\mat}{\mathbf} \begin{bmatrix} \mat A&\mat B^T\\ \mat B &-\mat C\end{bmatrix} =
        \begin{bmatrix} \mat I & \\ \mat B\mat A^{-1} & \mat I \end{bmatrix}
        \begin{bmatrix} \mat A & \\ & \mat S \end{bmatrix}
        \begin{bmatrix} \mat I & \mat A^{-1} \mat B^T \\ & \mat I \end{bmatrix} $$
  where $ \mathbf S = -\mathbf C -\mathbf B \mathbf A^{-1} \mathbf B^T $ 

# Spectral equivalence

Def: $A,B$ are *spectrally equivalent* if the eigenvalues $\lambda$ of $B^{-1}A$ satisfy $c_1 \leq \lambda \leq c_2$

- A common technique to show that two _operators_ are spectrally equivalent is to show instead that
    $$ c_1 \langle Bx, x\rangle \leq \langle Ax, x\rangle \leq c_2 \langle Bx, x\rangle $$ 
-  A simple example: $A= -\mathrm{div}\,\kappa \nabla u $ with $B=-\Delta u$
-  A much less simple example from CFD: $A=\mathrm{div}(\Delta)^{-1}\nabla$ and $P=I$ 

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square
from ngsolve.krylovspace import CGSolver
import pandas as pd

def getSystem(maxh, p, omega, preconditioner, use_equivalent=False):
    mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
    V = H1(mesh, order=p, dirichlet="left")
    u, v = V.TnT()
    mu = 1 + omega * sin(4 * pi * x) * sin(4 * pi * y)
    form = InnerProduct(mu * Grad(u), Grad(v))*dx #+ u * v * dx
    formP = InnerProduct(Grad(u), Grad(v))*dx #+ u * v * dx
    a = BilinearForm(form)
    aP_form = BilinearForm(formP) if use_equivalent else a 
    aP = Preconditioner(aP_form, preconditioner)
    a.Assemble()
    aP_form.Assemble()
    f = LinearForm(v*dx).Assemble()
    gf = GridFunction(V)
    return a, aP, f, gf, V

def getIts(a, aP, f, gf):
    inv = CGSolver(mat=a.mat, pre=aP.mat, printrates=False, maxiter=200) 
    gf.vec.data = inv * f.vec
    return inv.iterations

In [2]:
# Direct solver on spectrally equivalent operator
hs = [0.05, 0.02, 0.01, 0.0075, 0.005, 0.0025]

dofs = []
iterations_direct_no_equiv = []
iterations_direct_equiv = []
iterations_direct_no_equiv_big_omega = []
iterations_direct_equiv_big_omega = []
omega_small = 0.3
omega_big = 0.9
for h in hs:
    print("========= Running h={:1.4f}".format(h))
    a, aP, f, gf, V = getSystem(h, 1, omega_small, "direct", False)
    dofs.append(V.ndofglobal)
    # Direct, no equiv
    iterations_direct_no_equiv.append(getIts(a, aP, f, gf))
    # Direct, equiv
    a, aP, f, gf, V = getSystem(h, 1, omega_small, "direct", True)
    iterations_direct_equiv.append(getIts(a, aP, f, gf))
    # Same with big omegas
    a, aP, f, gf, V = getSystem(h, 1, omega_big, "direct", False)
    iterations_direct_no_equiv_big_omega.append(getIts(a, aP, f, gf))
    # Direct, equiv
    a, aP, f, gf, V = getSystem(h, 1, omega_big, "direct", True)
    iterations_direct_equiv_big_omega.append(getIts(a, aP, f, gf))
    
tab1 = pd.DataFrame({'DoFs':dofs, 
                    'Its direct': iterations_direct_no_equiv,
                    'Its direct, spect equiv': iterations_direct_equiv,
                    'Its direct, big $\omega$': iterations_direct_no_equiv_big_omega,
                    'Its preconditioned, big $\omega$': iterations_direct_equiv_big_omega})

========= Running h=0.0500
========= Running h=0.0200
========= Running h=0.0100


In [3]:
tab1 = tab1.style.set_caption('Number of CG iterations')
tab1

,DoFs,Its direct,"Its direct, spect equiv","Its direct, big $\omega$","Its preconditioned, big $\omega$"
0,511,2,15,2,44
1,3007,2,16,2,54
2,11786,2,16,2,55


In [4]:
# Same with optimal preconditioner (AMG)
iterations_precon_no_equiv = []
iterations_precon_equiv = []
omega = omega_big
for h in hs:
    print("========= Running h={:1.4f}".format(h))
    # Preconditioned, no equiv
    a, aP, f, gf, V = getSystem(h, 1, omega, "h1amg", False)
    iterations_precon_no_equiv.append(getIts(a, aP, f, gf))
    # Preconditioned, equiv
    a, aP, f, gf, V = getSystem(h, 1, omega, "h1amg", True)
    iterations_precon_equiv.append(getIts(a, aP, f, gf))
    
tab2 = pd.DataFrame({'DoFs':dofs, 
                    'Its direct': iterations_direct_no_equiv,
                    'Its preconditioned': iterations_precon_no_equiv,
                    'Its direct, spect equiv': iterations_direct_equiv,
                    'Its preconditioned, spectral equiv': iterations_precon_equiv})

========= Running h=0.0500
========= Running h=0.0200
========= Running h=0.0100


In [5]:
tab2 = tab2.style.set_caption('Number of CG iterations')
tab2

,DoFs,Its direct,Its preconditioned,"Its direct, spect equiv","Its preconditioned, spectral equiv"
0,511,2,19,15,45
1,3007,2,25,16,58
2,11786,2,30,16,66


# Multilevel methods

- Recursive
- Optimal
- Scalable
- Taylored for each specific problem...

      $$ \rho( \mathbf P_\text{ML}^{-1} \mathbf A) < C $$

# Multigrid method skeleton

We require a projection operator $\mathbf \Pi$ (extension is $\mathbf \Pi^T$)

Iteration $z = \mathrm{MG}[w, k]$:
1. Pre-smoothing: $ x \leftarrow x + \mathbf P_\text{pre}^{-1}(b - \mathbf A x) $
2. Coarse solve: $ x \leftarrow x + \mathbf \Pi^T \mathrm{MG}(\mathbf \Pi[b - \mathbf A x, k-1]) $
3. Post-smoothing: $ x \leftarrow x + \mathbf P_\text{post}^{-1}(b - \mathbf A x) $

- $\mathrm{MG}[\cdot, 0]$ is (usually) a direct solver
- $\mathbf P_{\star}$ is (usually) a smoother method (Jacobi, Gauss-Seidel, SOR, etc)

![Multigrid depiction](https://www.researchgate.net/profile/Piotr-Sypek-2/publication/4104513/figure/fig2/AS:669977363349515@1536746459391/The-examples-of-multigrid-schemes-The-V-cycle-is-the-basic-component-of-multigrid.png)

In [6]:
class MGPreconditioner(BaseMatrix):
    def __init__ (self, fes, level, mat, coarsepre):
        super().__init__()
        self.fes = fes
        self.level = level
        self.mat = mat
        self.coarsepre = coarsepre
        if level > 0:
            self.localpre = mat.CreateSmoother(fes.FreeDofs())
        else:
            self.localpre = mat.Inverse(fes.FreeDofs())

    def Mult (self, d, w):
        if self.level == 0:
            w.data = self.localpre * d
            return

        prol = self.fes.Prolongation().Operator(self.level)

        w[:] = 0
        self.localpre.Smooth(w,d)
        res  = d - self.mat * w
        w += prol @ self.coarsepre @ prol.T * res
        self.localpre.SmoothBack(w,d)


    def Shape (self):
        return self.localpre.shape
    def CreateVector (self, col):
        return self.localpre.CreateVector(col)



In [14]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))

fes = H1(mesh,order=1, dirichlet=".*", autoupdate=True)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx)
a.Assemble()
pre = MGPreconditioner(fes, 0, a.mat, None)
from ngsolve.krylovspace import CGSolver

dofs = []
iterations = []
levels = 9
for l in range(levels):
    mesh.Refine()
    a.Assemble()
    pre = MGPreconditioner(fes,l+1, a.mat, pre)
    inv = CGSolver(mat=a.mat, pre=pre, printrates=False, maxiter=200, tol=1e-8) 
    f = LinearForm(1*v*dx).Assemble()
    gf = GridFunction(fes)
    gf.vec.data = inv * f.vec
    dofs.append(fes.ndofglobal)
    iterations.append(inv.iterations)
tab3 = pd.DataFrame({'Dofs': dofs, 'Iterations': iterations})
tab3 = tab3.style.set_caption('CG+MG iterations')

,Dofs,Iterations
0,61,8
1,217,10
2,817,11
3,3169,11
4,12481,11
5,49537,11
6,197377,11
7,787969,11
8,3148801,11
